# Tarea 3 - Pregunta 3 - Calidad de un vino
---
# Matías Araya - 201173082-8
# Claudia Hazard - 201404523-9

# a)
Se cargan los dos dataset en un único dataframe y se agrega una columna indicando si
es vino tinto o blanco.

In [15]:
import pandas as pd
df_red = pd.read_csv("winequality-red.csv",sep=";") #Tinto
df_white = pd.read_csv("winequality-white.csv",sep=";") #Blanco
df = pd.concat([df_red,df_white], axis=0)
l=[]
for i in range(len(df_red)):
    l.append(0)
for i in range(len(df_white)):
    l.append(1)
df["tipo"]=l
print(len(df_red))
print(len(df_white))
print(max(df["quality"]))
df

1599
4898
9


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,tipo
0,7.4,0.700,0.00,1.90,0.076,11.0,34.0,0.99780,3.51,0.56,9.400000,5,0
1,7.8,0.880,0.00,2.60,0.098,25.0,67.0,0.99680,3.20,0.68,9.800000,5,0
2,7.8,0.760,0.04,2.30,0.092,15.0,54.0,0.99700,3.26,0.65,9.800000,5,0
3,11.2,0.280,0.56,1.90,0.075,17.0,60.0,0.99800,3.16,0.58,9.800000,6,0
4,7.4,0.700,0.00,1.90,0.076,11.0,34.0,0.99780,3.51,0.56,9.400000,5,0
5,7.4,0.660,0.00,1.80,0.075,13.0,40.0,0.99780,3.51,0.56,9.400000,5,0
6,7.9,0.600,0.06,1.60,0.069,15.0,59.0,0.99640,3.30,0.46,9.400000,5,0
7,7.3,0.650,0.00,1.20,0.065,15.0,21.0,0.99460,3.39,0.47,10.000000,7,0
8,7.8,0.580,0.02,2.00,0.073,9.0,18.0,0.99680,3.36,0.57,9.500000,7,0
9,7.5,0.500,0.36,6.10,0.071,17.0,102.0,0.99780,3.35,0.80,10.500000,5,0


El dataset consta 6497 datos de los cuales 1599 son de vino tinto y 4898 de vino blanco. Ademas tiene 13 columnas de los cuales 8 miden cantidad de algún elemento, ademas de los grados alcoholicos, la densidad, el pH, el tipo y lo que fue catalogado como su calidad siendo 9/10 la máxima calidad recibida.

# b)
Se aborda el problema como clasificación binaria cambiando la calidad a buena cuando es mayor a 5 y mala en otro caso. Luego se crean data de entrnamiento y prueba.

In [16]:
df['good_quality'] = [1 if q>5 else 0 for q in df.quality] #Se califica como buena calidad cuando la calidad es mayor que 5.
del df["quality"]
#Se desordenan los datos.
df = df.sample(frac=1,random_state=42)
#Se crean los data train y test
import numpy as np
msk = np.random.rand(len(df)) < 0.75
df_train = df[msk]
df_test = df[~msk]
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,tipo,good_quality
1504,7.0,0.170,0.74,12.80,0.045,24.0,126.0,0.99420,3.26,0.38,12.20,1,1
1419,7.7,0.640,0.21,2.20,0.077,32.0,133.0,0.99560,3.27,0.45,9.90,0,0
3162,6.8,0.390,0.34,7.40,0.020,38.0,133.0,0.99212,3.18,0.44,12.00,1,1
3091,6.3,0.280,0.47,11.20,0.040,61.0,183.0,0.99592,3.12,0.51,9.50,1,1
2433,7.4,0.350,0.20,13.90,0.054,63.0,229.0,0.99888,3.11,0.50,8.90,1,1
1297,7.2,0.530,0.14,2.10,0.064,15.0,29.0,0.99323,3.35,0.61,12.10,0,1
174,7.5,0.270,0.31,17.70,0.051,33.0,173.0,0.99900,3.09,0.64,10.20,1,0
3985,6.8,0.110,0.27,8.60,0.044,45.0,104.0,0.99454,3.20,0.37,9.90,1,1
561,9.0,0.440,0.49,2.40,0.078,26.0,121.0,0.99780,3.23,0.58,9.20,0,0
4347,7.1,0.230,0.30,2.60,0.034,62.0,148.0,0.99121,3.03,0.56,11.30,1,1


# c)
Entrene un solo Arbol de Clasificación de múltiples niveles para resolver el problema. Puede variar los ´
hiper-parámetros que prefiera, recuerde que las decisiones no pueden ser basadas mirando el conjunto
de pruebas. Debido al desbalanceo que se produce en las dos clases mida la m´etrica F1-score [5] sobre
el conjunto de entrenamiento y de pruebas.

In [40]:
x_train = df_train
y_train = np.asarray(df_train.good_quality.values).ravel().tolist()
x_test = df_test
y_test = np.asarray(df_test.good_quality.values).ravel().tolist()
from sklearn.tree import DecisionTreeClassifier as Tree
model= Tree()
model.set_params(criterion='gini',splitter='best')
model.fit(x_train, y_train)

from sklearn.metrics import f1_score
y_true =  y_train
y_pred =  model.predict(x_train)
f1_score(y_true, y_pred, average='binary') # average="micro" average="macro" average='weighted'


1.0

# d)
Entrene un ensamblador de árboles de múltiples niveles, mediante la técnica de Random Forest. Varíe la
cantidad de árboles de decisión utilizados en el ensamblado (n estimators), realice un gráfico resumen
del F1-score de entrenamiento y de pruebas en función de este hiper-parámetro.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=, max_depth=,n_jobs=-1)
from sklearn.metrics import f1_score
f1_score(y_true, y_pred)


# e)
 Entrene un ensamblador de árboles de múltiples niveles, mediante la técnica de AdaBoost. Varíe la
cantidad de árboles de decisión utilizados en el ensamblado (n estimators), realice un gráfico resumen
del F1-score de entrenamiento y de pruebas en función de este hiper-parámetro. Compare y analice con
la técnica utilizada en d).

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(base_estimator=Tree(max_depth=), n_estimators=)

# f)
Entrene alguna otra máquina de aprendizaje, elegida por usted, para resolver este problema. Elija los
hiper-parámetros que estime convenientes intentando aumentar el F1-score obtenido por los algoritmos
anteriores. Compare y analice estas 4 maneras de resolver el problema definido en b).

# g)
Defina un criterio para estimar la importancia de los distintos atributos en el ensamblado de Random Forest, implementelo sobre alguno de los ensambladores entrenados en d), haga un ranking de importancia de atributos >Es posible implementar este criterio sobre una técnica de boost como lo es AdaBoost?